In [1]:
import pandas as pd
import numpy as np



In [2]:
from preprocess import init_data
from preprocess import remove_cols

df = init_data('dataset.csv')

cols = list(df.select_dtypes(include=object))

df = remove_cols(df, cols)


In [3]:
df.shape

(62288, 6)

In [16]:
df.head()

,ID,Age,Commision (in value),Duration,Net Sales,Claim
0,45341,28,28.13,34,112.5,1
1,12958,37,12.95,53,37.0,0
2,18233,27,0.00,28,13.0,0
3,31742,36,0.00,1,34.0,0
4,14381,26,23.76,33,39.6,0


In [3]:
from feature_extraction import select_features_rfecv

df = pd.get_dummies(df, drop_first=True)

X = df.drop('Claim', axis=1)
y = df['Claim']

X = X.replace({True : 1, False : 0})

features = select_features_rfecv(X, y, n_features=2)

In [4]:
features

['ID', 'Net Sales']

In [5]:
from models import ml_models

models = ml_models(df)

best_model, best_metric, X_test, y_test =  models.get_best_model_in_test()

print(f"Лучшая модель: {best_model}, F1-Score: {best_metric}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 8664, number of negative: 34937
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000895 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1058
[LightGBM] [Info] Number of data points in the train set: 43601, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.198711 -> initscore=-1.394370
[LightGBM] [Info] Start training from score -1.394370
Лучшая модель: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_b

In [6]:
best_model, best_metric

(XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=None, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=None, n_jobs=None,
               num_parallel_tree=None, random_state=None, ...),
 0.8839190628328009)

In [7]:

best_tuned_model, tuned_params = models.tune_best_model_with_optuna()
print(f"Тюнинг завершён! Лучшие параметры: {tuned_params} лучшая модель:{best_tuned_model}")


[I 2024-12-22 15:39:26,942] A new study created in memory with name: no-name-7c90b06a-d354-4728-9bff-236d2f2e600e


[I 2024-12-22 15:39:27,424] Trial 0 finished with value: 0.6359422237151495 and parameters: {'eta': 0.1928485255113524, 'n_estimators': 382, 'max_depth': 4, 'subsample': 0.7843311032100899, 'colsample_bytree': 0.9433468250829662, 'gamma': 3.784049681669437}. Best is trial 0 with value: 0.6359422237151495.
[I 2024-12-22 15:39:27,674] Trial 1 finished with value: 0.9655407710678949 and parameters: {'eta': 0.19038210120253646, 'n_estimators': 99, 'max_depth': 14, 'subsample': 0.8201901892992296, 'colsample_bytree': 0.8220938699300417, 'gamma': 1.892381378796465}. Best is trial 1 with value: 0.9655407710678949.
[I 2024-12-22 15:39:28,384] Trial 2 finished with value: 0.29808846615569307 and parameters: {'eta': 0.0033695696566718885, 'n_estimators': 393, 'max_depth': 4, 'subsample': 0.5131730840845061, 'colsample_bytree': 0.6970091843049322, 'gamma': 4.521165920457962}. Best is trial 1 with value: 0.9655407710678949.
[I 2024-12-22 15:39:30,310] Trial 3 finished with value: 0.0 and parameter

Тюнинг завершён! Лучшие параметры: {'eta': 0.03461451503945023, 'n_estimators': 315, 'max_depth': 13, 'subsample': 0.8421221791598954, 'colsample_bytree': 0.8522003082377558, 'gamma': 0.2621900618714067} лучшая модель:XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8522003082377558, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eta=0.03461451503945023, eval_metric=None, feature_types=None,
              gamma=0.2621900618714067, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=13, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=315, n_jobs=None,
              num_para

In [8]:
from get_dataset import evaluate_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


res_df, mse = evaluate_model(best_tuned_model, X_test, y_test, mean_squared_error)



In [9]:
res_df

,ID,Age,Commision (in value),Duration,Net Sales,y_real,y_predict
44271,41878,36,0.00,45,56.00,0,0
6178,47514,50,0.00,19,10.00,0,0
59156,33839,36,0.00,25,39.00,0,0
47561,52364,39,7.70,27,22.00,1,1
21363,53624,59,35.64,31,59.40,0,0
...,...,...,...,...,...,...,...
44928,24232,27,0.00,61,90.00,0,0
7452,20110,47,46.96,365,187.85,1,1
45585,48224,56,12.13,17,48.50,1,1
51333,6710,43,17.71,21,70.85,0,0


In [10]:
mse

0.004548616685396265

In [5]:
from blending import Blend

bl = Blend(df)

f1, final_preds = bl.fit_and_blend()

[I 2024-12-22 00:20:28,366] A new study created in memory with name: no-name-b83e175a-b791-47e0-850f-15245da2a757


[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] feature_fraction is set=0.9043387553759766, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9043387553759766
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] feature_fraction is set=0.9043387553759766, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9043387553759766
[LightGBM] [Info] Number of positive: 8664, number of negative: 34937
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001831 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1056
[LightGBM] [Info] Number of data points in the train set: 43601, number of used features: 5
[Light

[I 2024-12-22 00:20:29,107] Trial 0 finished with value: 0.0 and parameters: {'num_leaves': 62, 'learning_rate': 0.0011466717315355376, 'n_estimators': 334, 'max_depth': 4, 'min_data_in_leaf': 100, 'feature_fraction': 0.9043387553759766}. Best is trial 0 with value: 0.0.


[LightGBM] [Warning] min_data_in_leaf is set=29, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=29
[LightGBM] [Warning] feature_fraction is set=0.6422542211845168, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6422542211845168
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] min_data_in_leaf is set=29, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=29
[LightGBM] [Warning] feature_fraction is set=0.6422542211845168, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6422542211845168
[LightGBM] [Info] Number of positive: 8664, number of negative: 34937
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001401 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1056
[LightGBM] [Info] Number of data points in the train set: 43601, number of used features: 5
[LightGBM]

[I 2024-12-22 00:20:30,878] Trial 1 finished with value: 0.0 and parameters: {'num_leaves': 132, 'learning_rate': 0.0012643608150778559, 'n_estimators': 420, 'max_depth': 8, 'min_data_in_leaf': 29, 'feature_fraction': 0.6422542211845168}. Best is trial 0 with value: 0.0.


...

[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] feature_fraction is set=0.7438056689885775, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7438056689885775
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] feature_fraction is set=0.7438056689885775, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7438056689885775
[LightGBM] [Info] Number of positive: 8664, number of negative: 34937
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001107 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1056
[LightGBM] [Info] Number of data points in the train set: 43601, number of used features: 5
[LightGBM]

[I 2024-12-22 00:21:30,523] A new study created in memory with name: no-name-1bd32ab5-f19b-40c8-afd9-39b88d878007


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2024-12-22 00:21:32,348] Trial 0 finished with value: 0.0 and parameters: {'eta': 0.0018733656201595573, 'n_estimators': 311, 'max_depth': 15, 'subsample': 0.5025494824573371, 'colsample_bytree': 0.8941952515203582, 'gamma': 3.1817228132710045}. Best is trial 0 with value: 0.0.
[I 2024-12-22 00:21:33,640] Trial 1 finished with value: 0.4096065225097483 and parameters: {'eta': 0.0033636767531928227, 'n_estimators': 311, 'max_depth': 11, 'subsample': 0.6213931420750842, 'colsample_bytree': 0.7217062097666871, 'gamma': 4.518049508024241}. Best is trial 1 with value: 0.4096065225097483.
[I 2024-12-22 00:21:34,098] Trial 2 finished with value: 0.8860414071305689 and parameters: {'eta': 0.14724737369050872, 'n_estimators': 359, 'max_depth': 11, 'subsample': 0.71785936155238, 'colsample_bytree': 0.7393885490680945, 'gamma': 2.8402025051137496}. Best is trial 2 with value: 0.8860414071305689.
[I 2024-12-22 00:21:34,786] Trial 3 finished with value: 0.10923276983094929 and parameters: {'eta'

[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] feature_fraction is set=0.7438056689885775, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7438056689885775

Блендинг complete! F1-score: 0.9854545454545455
